In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Papillary_Cell_Carcinoma/GSE26574'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"An antioxidant response phenotype is shared between hereditary and sporadic type 2 papillary renal cell carcinoma"
!Series_summary	"Fumarate hydratase (FH) mutation causes hereditary type 2 papillary renal cell carcinoma (HLRCC, Hereditary Leiomyomatosis and Renal Cell Cancer (MM ID # 605839)). The main effect of FH mutation is fumarate accumulation. The current paradigm posits that the main consequence of fumarate accumulation is HIF-a stabilization. Paradoxically, FH mutation differs from other HIF-a stabilizing mutations, such as VHL and SDH mutations, in its associated tumor types. We identified that fumarate can directly up-regulate antioxidant response element (ARE)-controlled genes. We demonstrated that AKR1B10 is an ARE-controlled gene and is up-regulated upon FH knockdown as well as in FH-null cell lines. AKR1B10 overexpression is also a prominent feature in both hereditary and sporadic PRCC2. This phenotype better explains the similariti

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine gene expression data availability
is_gene_available = True

# Variables and their corresponding rows in the sample characteristics dictionary
trait_row = 0  # Assuming 'disease state' column is relevant
age_row = None  # Assuming no data found in provided characteristics
gender_row = None  # Assuming no data found in provided characteristics

# Conversion Functions
def convert_trait(value):
    mapping = {
        'normal_tissue_from_ccRCC_patient': 0,
        'normal_tissue_from_FH_patient': 0,
        'ccRCC': None,
        'Chromophobe': None,
        'Pap_type1': None,
        'Pap_type2': 1,
        'HLRCC': 1
    }
    trait_value = value.split(':')[1].strip()
    return mapping.get(trait_value, None)

def convert_age(value):
    return None  # No age data available to convert

def convert_gender(value):
    return None  # No gender data available to convert

# Save cohort information
save_cohort_info('GSE26574', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    try:
        selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Papillary_Cell_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
        csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data/GSE26574.csv'
        selected_clinical_data.to_csv(csv_path)
        print(preview_df(selected_clinical_data))
    except Exception as e:
        print(f"Error during clinical feature extraction: {e}")


{'GSM655513': [0], 'GSM655514': [0], 'GSM655515': [0], 'GSM655516': [0], 'GSM655517': [0], 'GSM655518': [0], 'GSM655519': [0], 'GSM655520': [0], 'GSM655521': [None], 'GSM655522': [None], 'GSM655523': [None], 'GSM655524': [None], 'GSM655525': [None], 'GSM655526': [None], 'GSM655527': [None], 'GSM655528': [None], 'GSM655529': [None], 'GSM655530': [None], 'GSM655531': [None], 'GSM655532': [None], 'GSM655533': [None], 'GSM655534': [None], 'GSM655535': [None], 'GSM655536': [None], 'GSM655537': [None], 'GSM655538': [None], 'GSM655539': [None], 'GSM655540': [None], 'GSM655541': [None], 'GSM655542': [None], 'GSM655543': [None], 'GSM655544': [None], 'GSM655545': [None], 'GSM655546': [None], 'GSM655547': [None], 'GSM655548': [None], 'GSM655549': [None], 'GSM655550': [None], 'GSM655551': [None], 'GSM655552': [None], 'GSM655553': [None], 'GSM655554': [None], 'GSM655555': [None], 'GSM655556': [None], 'GSM655557': [None], 'GSM655558': [None], 'GSM655559': [None], 'GSM655560': [None], 'GSM655561': [1

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '9', '10', '12', '13', '14', '15', '16', '18', '19', '20',
       '21', '22', '23', '24', '25', '26', '27', '28'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '10', '100', '1000', '10000'], 'CHR': ['19', '8', '20', '18', '1'], 'ORF': ['A1BG', 'NAT2', 'ADA', 'CDH2', 'AKT3'], 'GENE_ID': [1.0, 10.0, 100.0, 1000.0, 10000.0]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'ORF'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE26574.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_Papillary_Cell_Carcinoma')

# 4. Save the cohort information.
json_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json'
save_cohort_info('GSE26574', json_path, True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/GSE26574.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Kidney_Papillary_Cell_Carcinoma', the least common label is '0.0' with 10 occurrences. This represents 37.04% of the dataset.
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is fine.

